# CC3001 Otoño 2024 Tarea 3  Felipe González Poblete

# Calculadora

### Profesores
Sección 1 Patricio Poblete, Nelson Baloian

Atención: la forma de entrega tiene requisitos levemente distintos esta vez




# Enunciado

Esta tarea consiste en desarrollar una calculadora de expresiones matemáticas al estilo Matlab o Maple, pero con capacidad de procesar un conjunto de comandos bien restringido. Algunos ejemplos de los comandos que acepta y de lo que debe imprimir en la salida van a continuación:

|Entrada (lo que el usuario escribe)   | Salida(lo que imprime el programa)  |
|---|---|
|n=5   | n=5 |
|   | 5 |
|hanoi=2^n-1    |hanoi = 2^n-1  |
|    |31   |
|var_1 = 23 - 13 + hanoi * 2    |var_1 = 23 - 13 + hanoi * 2   |
|    | 82   |
|h2 = hanoi /2   |h2 = hanoi /2 |
|   | 15   |
|var_2 = (2+n)*2   |var_2 = (2+n)*2    |
|   | 14   |
|var_3 = ((1+n)/2 +3)^2   |var_3 = ((1+n)/2 +3)^2   |
|   | 36   |
|n   |n   |
|  | 5   |

A continuación definimos más en detalle lo que se debe implementar:



1.   Cada comando es de la forma "variable=expresión". El efecto es que primero se debe imprimir el comando y luego se debe calcular la expresión de la derecha, guardar el resultado en la variable de la izquierda e imprimir el resultado en la salida. Si se omite desde el "=" hacia adelante (como en el último ejemplo), solo se imprime el valor de la variable. Las variables comienzan con letra y continúan con letras, dígitos y el signo "_".
2.   Los operadores permitidos son "+","-","*", "/", "^".
3.   Solo se trabaja con números enteros y todas las operaciones dan resultado entero (incluyendo la división, que trunca).
4.   Se puede usar paréntesis, y si no los hay, la expresión se evalúa estrictamente de izquierda a derecha. Eso explica el resultado que se obtiene para "var_1", no hay prioridad de operadores. Pueden venir paréntesis anidados (ver ejemplos)

  **Nota**: Se puede optar por el 70% de la nota si se implementa todo lo solicitado salvo el manejo de paréntesis. En este caso, las expresiones se evaluarán estrictamente de izquierda a derecha sin considerar la prioridad de operadores, y no se permitirá el uso de paréntesis en las expresiones.

5.   Si se utiliza una variable que aún no ha sido definida, se debe dar un error

      ERROR: variable indefinida "..."

6.   Si la sintaxis no es correcta, se debe imprimir en la salida un mensaje de la forma

      ERROR: al procesar "..."

      donde el string que se imprime es la parte de la entrada desde el punto del error hacia adelante.


Antes de empezar a escribir código, usted debe dibujar un diagrama de estados que describa la estructura de la entrada. Si le resulta más conveniente, pueden ser varios diagramas (por ejemplo, uno que describa la estructura de una variable, otro la de un número, otra la de un comando). Luego, a partir de ahí escriba el código que implementa esos diagramas. **Importante:** Su código debe ir procesando de un caracter a la vez, para este procesamiento no se puede utilizar funciones de Python que operen sobre strings de largo mayor que uno. Se recomienda tener una función que reciba un comando en un string y lo procese, y otra que reciba una lista de comandos y vaya invocando a la función antes mencionada para procesarlos.

Para almacenar los nombres y valores de las variables, está permitido el uso de un diccionario de Python.

En su entrega debe describir brevemente el problema, luego describir la estrategia de solución haciendo referencia a su(s) diagrama(s) de estados, y a continuación el código ejecutable respectivo, agregando todas las explicaciones necesarias para que se entienda.



In [1]:
def estado_leer_comando(op_char,k):
    """recibe un string de largo 1 y retorna el estado correspondiente
    """
    estado = 'lectura_comando'
    separadores = ' +-*/^'
    if k == 0:
        estado = 'imprimir_variable'
    
    if op_char.isalpha():
        estado = 'lectura_variable'

    elif op_char.isdecimal():
        estado = 'lectura_numero'

    elif op_char in separadores:
        estado = 'guardar_operador'

    else:
        estado = 'ERROR: al procesar'
    
    return estado



In [2]:
def lectura_variable(operacion, dicc_var):
    """Recibe un string de la forma '{variable}={expresion}'
    y entrega una tupla, conteniendo el nombre de la variable
    y el índice donde se encuentra el signo '='
    """
    k = 0
    var = ''
    simbolos_prohibidos = ' +-*/^'
        
    while k < len(operacion) and len(operacion) != 0:
        if operacion in dicc_var:
            return var, -1
        
        if (operacion[k] == '=' and k != 0):
            return var,k
        
        elif operacion[k] not in simbolos_prohibidos:
            var += operacion[k]
        
        k += 1
    return var, k

In [3]:
def evaluar(base, numero, operador):
    """ operadores permitidos: ' +-*/^'
    Recibe un entero y un entero base y entrega base ~ numero, 
    donde ~ es cualquiera de los operadores permitidos
    """    
    if operador == '+' or operador == '':
        return base + numero
    elif operador == '-':
        return base - numero
    elif operador == '*':
        return base * numero
    elif operador == '/':
        return base // numero
    elif operador == '^':
        return base ** numero
    else:
        return ''

In [4]:
def procesar_expresion(operacion, dicc_var, k2):
    """
    Recibe un string de la forma '{variable}={expresion}' y comienza a leer desde la 
    (k+1)-ésima posición.
    devuelve un número.
    """
    separadores = ' +-*/^'
    k = k2 + 1
    variable = ''
    operador = ''
    valor = 0
    estado = 'lectura_comando'
    N = len(operacion)
    
    while estado != 'success':
        if k >= N:
            estado = 'success'
        
        if estado == 'lectura_comando':
            variable = ''
            estado = estado_leer_comando(operacion[k], k)            
        
        if estado == 'lectura_variable':
                        
            if operacion[k] not in separadores:
                variable += operacion[k]
                k += 1
            
            if k >= len(operacion) or operacion[k] in separadores:
                if variable in dicc_var:
                    valor_variable = dicc_var[variable]
                    valor = evaluar(valor, valor_variable, operador)
                    estado = 'lectura_comando'
                else:
                    estado = 'ERROR: variable indefinida'
                    
                    
        if estado == 'lectura_numero':
            if operacion[k].isalpha():
                estado = 'ERROR: al procesar'
                
            elif operacion[k] not in separadores :
                variable += operacion[k]
                k += 1
                
                if k >= N:
                    estado = 'success'
                    valor = evaluar(valor,int(variable),operador)
                
                elif operacion[k] in separadores:
                    valor = evaluar(valor,int(variable),operador)
                    estado = 'guardar_operador'
                
                
        if estado == 'imprimir_variable':
            valor = dicc_var[operacion]
            estado = 'procesar_comando'
            return valor
        
        if estado == 'guardar_operador':
            if operacion[k] != ' ':
                operador = operacion[k]
            estado = 'lectura_comando'
            k += 1
                        
        if estado == 'success':
            return valor
            
        if estado == 'ERROR: variable indefinida':
            return f'ERROR: variable indefinida {variable}'
        
        if estado == 'ERROR: al procesar':
            return f'ERROR: variable indefinida {variable}'   
    return valor

In [ ]:
def lectura_parentesis(operacion, dicc_var, k2):
    """recibe una expresion con parentesis y va evaluando segun
    los parentesis que recibe, guardando los valores de los
    parentesis anidados
    o al menos esa es la idea jaja"""
    
    dic_parentesis = {}
    
    return valor

In [5]:

def procesar_comando(comando, dicc_var):
    """Esta función recibe un comando en string y el diccionario de variables. Con él,
    procesa el comando, imprime el resultado de la expresion
    y posiblemente modifica el diccionario. La función retorna el diccionario"""
    var, k = lectura_variable(comando, dicc_var)
    value = procesar_expresion(comando,dicc_var, k)
    
    if type(value) != str:
        dicc_var[var] = value 
    print(value)
    return dicc_var


In [6]:
def calculadora(lista_comandos):
    """Este diccionario almacena las variables que se vayan definiendo en la calculadora
    Este diccionario se inicializa cuando se usa la calculadora"""
    vars = {}
    n = len(lista_comandos)

  # Se procesan todos los comandos de la lista (lista de string)
    for i, comando in enumerate(lista_comandos):
        if comando != '':
            print(f'{comando}')
            vars = procesar_comando(comando, vars)

In [7]:
lista1 = ["n=5","hanoi=2^n-1","var_1 = 23 - 13 + hanoi * 2","h2 = hanoi /2","","n"]
lista2 = ["n=5","hanoi=2^n-1","var_1 = 23 - 13 + hanoi2 * 2","h2 = hanoi /2","","n"]

L = [lista1,lista2]

for i, lista in enumerate(L):
    ej = f' EJEMPLO {i+1} '
    print(f"\033[95m\n{ej.center(40,'=')}\033[00m")
    calculadora(lista)


============== EJEMPLO 1 ===============
n=5
5
hanoi=2^n-1
31
var_1 = 23 - 13 + hanoi * 2
82
h2 = hanoi /2
15
n
5

============== EJEMPLO 2 ===============
n=5
5
hanoi=2^n-1
31
var_1 = 23 - 13 + hanoi2 * 2
ERROR: variable indefinida hanoi2
h2 = hanoi /2
15
n
5


El resultado del ejemplo anterior debería imprimir en pantalla

   n=5

   5

   hanoi=2^n-1
   
   31

   var_1 = 23 - 13 + hanoi * 2
   
   82

   h2 = hanoi /2
   
   15

   n
   
   5



## Ejemplo 2

El resultado del ejemplo anterior debería imprimir en pantalla

n=5

5

hanoi=2^n-1

31

var_1 = 23 - 13 + hanoi2 * 2

ERROR: variable indefinida "hanoi2"

h2 = hanoi /2

15

n

5

## ¿Qué hay que entregar?

En esta tarea Usted DEBE entregar, además del programa que implementa la solución, el dibujo que muestra el diagrama de estados que implementa su programa. Esto debe hacerse subiendo un archivo separado (cualquier tipo de documento que pueda contener una imagen o un dibujo, por ejemplo, un PDF, el escaneo de una página donde dibujó a mano el diagrama, etc.).

 Tal como se ha hecho en tareas anteriores, para su programa usted debe crear una copia de este documento (del menú seleccionar "Archivo" o "File" y de ahi la opción "guardar copia en Drive" o "save a copy in Drive"). Este nuevo documento (notebook de Colab) es el que debe modificar para luego compartirlo con la opción "Cualquier usuario de internet con este enlace puede ver esto" (o “Anyone on the internet with this link can view”). Esta opción se encuentra disponible en la parte superior derecha de su notebook, en el botón "Compartir" (o "Share"). Una vez que tenga disponible el enlace, colocarlo en en el lugar de comentarios de la entrega de su tarea (en este caso, como comentario del archvo con el diagrama). No olvide colocar su nombre dentro del notebook.

 Se puede optar por el 70% de la nota si se implementa todo lo solicitado en el enunciado excepto el manejo de paréntesis. Esto implica que el cálculo de las expresiones deberá realizarse estrictamente de izquierda a derecha, sin considerar la precedencia de operadores. En este caso, las expresiones no incluirán paréntesis anidados ni simples, por lo que no es necesario diseñar un sistema que los procese. El resto de las funcionalidades, como la definición de variables, los operadores permitidos y la detección de errores, deben implementarse completamente.